Import

In [21]:
import requests
import pandas as pd
import xml.etree.ElementTree as ET
from bs4 import BeautifulSoup


In [22]:
soup = ""
with open("data.xml", "r", encoding="utf-8") as file:
    content = file.read()
    soup = BeautifulSoup(content, "xml")
    file.close()


#ProductTree

In [23]:
library = soup.find('ProductTree')
productdb = {'ProductID':[], 'ProductName':[], 'Type':[], 'Name':[]}


def process_node(node, productdb):

    if node.name is not None:
        if 'ProductID' in node.attrs:
            productdb['ProductID'].append(node.attrs['ProductID'])
            productdb['ProductName'].append(node.text)
        
        if 'Type' not in node.attrs and node.name != 'ProductTree':
            if 'Type' in node.parent.attrs:
                 productdb['Type'].append(node.parent.attrs['Type'])
                 productdb['Name'].append(node.parent.attrs['Name'])
            else:
                productdb['Type'].append(None)
                productdb['Name'].append(None)

        for child in node.children:
            process_node(child, productdb)


process_node(library, productdb)
productdb = pd.DataFrame(productdb)
productdb.to_excel('data.xlsx', index=False)
print(productdb)


    ProductID                                        ProductName  \
0       11568         Windows 10 Version 1809 for 32-bit Systems   
1       11569      Windows 10 Version 1809 for x64-based Systems   
2       11570    Windows 10 Version 1809 for ARM64-based Systems   
3       11571                                Windows Server 2019   
4       11572     Windows Server 2019 (Server Core installation)   
..        ...                                                ...   
177     12139                                CBL Mariner 2.0 x64   
178     12140                                CBL Mariner 2.0 ARM   
179      9312  Windows Server 2008 for 32-bit Systems Service...   
180      9318  Windows Server 2008 for x64-based Systems Serv...   
181      9344  Windows Server 2008 for x64-based Systems Serv...   

               Type     Name  
0    Product Family  Windows  
1    Product Family  Windows  
2    Product Family  Windows  
3    Product Family  Windows  
4    Product Family  Windows

#Vulnerability

In [24]:

# vulnerability_db = {"Ordinal":[], "CVE":[], "Title":[]}
# vulnerability_db_status = {"VulnerabilityFK":[], "StatusType": [], "ProductID":[]}
# vulnerability_db_notes = {"VulnerabilityFK":[], "Title":[], "Type":[], "Ordinal":[], "Note":[]}
# vulnerability_db_threats = {"VulnerabilityFK":[], "Type":[], "Description":[], "ProductID":[]}
# vulnerability_db_score_set = {"VulnerabilityFK":[],"BaseScore":[], "TemporalScore":[], "Vector":[], 'ProductID':[]}
# vulnerability_db_acknowledgment= {"VulnerabilityFK":[], "Name":[], "URL":[]}
# vulnerability_db_revision = {"VulnerabilityFK":[], "Number":[], 'Date':[], 'Description':[]}
# # vulnerability_db_Remediation = {"VulnerabilityFK":[], 'Type':[], 'Description':[], "URL":[], 'ProductID':[], 
# def vulnerability(node, vulnerability_db):
#     if node.name is not None:

#         def get_x_parent(res, x):
#             for i in range(x):
#                 res = res.parent
#             return res
# ### Vulnerability
#         if 'Ordinal' in node.attrs and node.name == 'Vulnerability':
#             vulnerability_db['Ordinal'].append(node.attrs['Ordinal'])

#             if node.find('Title').text == '':
#                  vulnerability_db['Title'].append(None)
#             else:
#                 vulnerability_db['Title'].append(node.text)
            
#             vulnerability_db['CVE'].append(node.find('CVE').text)

# ### Status
#         if node.name == 'ProductID' and node.parent.name == "Status":
#             vulnerability_db_status['ProductID'].append(node.text)
#             vulnerability_db_status['StatusType'].append(node.parent.attrs['Type'])
#             vulnerability_db_status['VulnerabilityFK'].append(len(vulnerability_db['Ordinal'])-1)

# ### Note


#         if node.name == 'Note' and node.parent.name == "Notes":
#             vulnerability_db_notes['VulnerabilityFK'].append(len(vulnerability_db['Ordinal'])-1)
#             vulnerability_db_notes["Title"].append(node.attrs['Title'])
#             vulnerability_db_notes["Type"].append(node.attrs['Type'])
#             vulnerability_db_notes["Ordinal"].append(node.attrs['Ordinal'])
#             vulnerability_db_notes["Note"].append(node.text)
# ### Threats
#         if node.name == 'Threat' and node.parent.name == "Threats":
#             vulnerability_db_threats['VulnerabilityFK'].append(len(vulnerability_db['Ordinal'])-1)
#             vulnerability_db_threats['Type'].append(node.attrs['Type'])

#             if node.find('ProductID'):
                
#                 for i in node.children:
#                     if i.name == 'ProductID': 
#                         vulnerability_db_threats['ProductID'].append(i.text)   

#                     if i.name == 'Description' and i.text != '': 
#                         vulnerability_db_threats['Description'].append(i.text)
                    
#                     if i.name == 'Description' and i.text == '': 
#                         vulnerability_db_threats['Description'].append(None)
#             else:
#                 vulnerability_db_threats['ProductID'].append(None) 
#                 vulnerability_db_threats['Description'].append(node.find("Description").text)
# ### Score Set
#         if node.name == 'ScoreSet' and node.parent.name == "CVSSScoreSets":
#             vulnerability_db_score_set['VulnerabilityFK'].append(len(vulnerability_db['Ordinal'])-1)
#             vulnerability_db_score_set['BaseScore'].append(node.find('BaseScore').text)
#             vulnerability_db_score_set['TemporalScore'].append(node.find('TemporalScore').text)
#             vulnerability_db_score_set['Vector'].append(node.find('Vector').text)
#             vulnerability_db_score_set['ProductID'].append(node.find('ProductID').text)

# ### Acknowledgment     
#         if node.name == 'Acknowledgment' and node.parent.name =='Acknowledgments':
#             vulnerability_db_acknowledgment['VulnerabilityFK'].append(len(vulnerability_db['Ordinal'])-1)

#             if node.find("Name"):

#                 if node.find('Name').text != '':
#                     vulnerability_db_acknowledgment['Name'].append(node.find('Name').text)
#                 else:
#                     vulnerability_db_acknowledgment['Name'].append(None)
            
#             if node.find("URL"):
                
#                 if node.find('URL').text != '': 
#                     vulnerability_db_acknowledgment["URL"].append(node.find('URL').text)
#                 else:
#                     vulnerability_db_acknowledgment["URL"].append(None)
            
# ### Revision
#         if node.name == 'Revision' and node.parent.name =='RevisionHistory':
#             vulnerability_db_revision['VulnerabilityFK'].append(len(vulnerability_db['Ordinal'])-1)
#             vulnerability_db_revision['Number'].append(node.find('Number').text)
#             vulnerability_db_revision['Date'].append(node.find('Date').text)
#             vulnerability_db_revision['Description'].append(node.find('Description').text)



#         for child in node.children:
#             vulnerability(child, vulnerability_db)

# lib = soup.find('cvrfdoc')
# lib = lib.children
# for child in lib:
#     if child.name == "Vulnerability":
#         vulnerability(child, vulnerability_db)
        
# vulnerability_db = pd.DataFrame(vulnerability_db)
# vulnerability_db_status = pd.DataFrame(vulnerability_db_status)
# vulnerability_db_notes = pd.DataFrame(vulnerability_db_notes)
# vulnerability_db_threats = pd.DataFrame(vulnerability_db_threats)
# vulnerability_db_score_set = pd.DataFrame(vulnerability_db_score_set)
# vulnerability_db_acknowledgment = pd.DataFrame(vulnerability_db_acknowledgment)
# vulnerability_db_revision = pd.DataFrame(vulnerability_db_revision)
# print(vulnerability_db_revision)
# # print(len(vulnerability_db_acknowledgment["VulnerabilityFK"]), len(vulnerability_db_acknowledgment["Name"]))


In [25]:
vulnerability_db = {"Ordinal":[], "CVE":[], "Title":[]}
vulnerability_db_status = {"VulnerabilityFK":[], "StatusType": [], "ProductID":[]}
vulnerability_db_notes = {"VulnerabilityFK":[], "Title":[], "Type":[], "Ordinal":[], "Note":[]}
vulnerability_db_threats = {"VulnerabilityFK":[], "Type":[], "Description":[], "ProductID":[]}
vulnerability_db_score_set = {"VulnerabilityFK":[],"BaseScore":[], "TemporalScore":[], "Vector":[], 'ProductID':[]}
vulnerability_db_acknowledgment= {"VulnerabilityFK":[], "Name":[], "URL":[]}
vulnerability_db_revision = {"VulnerabilityFK":[], "Number":[], 'Date':[], 'Description':[]}

def process_vulnerability(node, vulnerability_db):
    vulnerability_db['Ordinal'].append(node.get('Ordinal'))
    vulnerability_db['Title'].append(node.find('Title').text if node.find('Title').text else None)
    vulnerability_db['CVE'].append(node.find('CVE').text)

def process_status(node, vulnerability_db_status, vulnerability_index):
    vulnerability_db_status['ProductID'].append(node.text)
    vulnerability_db_status['StatusType'].append(node.parent.get('Type'))
    vulnerability_db_status['VulnerabilityFK'].append(vulnerability_index)

def process_notes(node, vulnerability_db_notes, vulnerability_index):
    vulnerability_db_notes['VulnerabilityFK'].append(vulnerability_index)
    vulnerability_db_notes['Title'].append(node.get('Title'))
    vulnerability_db_notes['Type'].append(node.get('Type'))
    vulnerability_db_notes['Ordinal'].append(node.get('Ordinal'))
    vulnerability_db_notes['Note'].append(node.text)

def process_threats(node, vulnerability_db_threats, vulnerability_index):
    vulnerability_db_threats['VulnerabilityFK'].append(vulnerability_index)
    vulnerability_db_threats['Type'].append(node.get('Type'))
    product_id = node.find('ProductID')
    vulnerability_db_threats['ProductID'].append(product_id.text if product_id else None)
    description = node.find('Description')
    vulnerability_db_threats['Description'].append(description.text if description else None)

def process_score_set(node, vulnerability_db_score_set, vulnerability_index):
    vulnerability_db_score_set['VulnerabilityFK'].append(vulnerability_index)
    vulnerability_db_score_set['BaseScore'].append(node.find('BaseScore').text)
    vulnerability_db_score_set['TemporalScore'].append(node.find('TemporalScore').text)
    vulnerability_db_score_set['Vector'].append(node.find('Vector').text)
    vulnerability_db_score_set['ProductID'].append(node.find('ProductID').text)

def process_acknowledgment(node, vulnerability_db_acknowledgment, vulnerability_index):
    vulnerability_db_acknowledgment['VulnerabilityFK'].append(vulnerability_index)
    name = node.find('Name')
    vulnerability_db_acknowledgment['Name'].append(name.text if name and name.text else None)
    url = node.find('URL')
    vulnerability_db_acknowledgment['URL'].append(url.text if url and url.text else None)

def process_revision(node, vulnerability_db_revision, vulnerability_index):
    vulnerability_db_revision['VulnerabilityFK'].append(vulnerability_index)
    vulnerability_db_revision['Number'].append(node.find('Number').text)
    vulnerability_db_revision['Date'].append(node.find('Date').text)
    vulnerability_db_revision['Description'].append(node.find('Description').text)


def vulnerability(node, vulnerability_db):
    if node.name is None:
        return
    
    vulnerability_index = len(vulnerability_db['Ordinal']) - 1

    if node.name == 'Vulnerability' and 'Ordinal' in node.attrs:
        process_vulnerability(node, vulnerability_db)
    elif node.name == 'ProductID' and node.parent.name == "Status":
        process_status(node, vulnerability_db_status, vulnerability_index)
    elif node.name == 'Note' and node.parent.name == "Notes":
        process_notes(node, vulnerability_db_notes, vulnerability_index)
    elif node.name == 'Threat' and node.parent.name == "Threats":
        process_threats(node, vulnerability_db_threats, vulnerability_index)
    elif node.name == 'ScoreSet' and node.parent.name == "CVSSScoreSets":
        process_score_set(node, vulnerability_db_score_set, vulnerability_index)
    elif node.name == 'Acknowledgment' and node.parent.name == 'Acknowledgments':
        process_acknowledgment(node, vulnerability_db_acknowledgment, vulnerability_index)
    elif node.name == 'Revision' and node.parent.name == 'RevisionHistory':
        process_revision(node, vulnerability_db_revision, vulnerability_index)

    for child in node.children:
        vulnerability(child, vulnerability_db)


lib = soup.find('cvrfdoc').children
for child in lib:
    if child.name == "Vulnerability":
        vulnerability(child, vulnerability_db)


vulnerability_db = pd.DataFrame(vulnerability_db)
vulnerability_db_status = pd.DataFrame(vulnerability_db_status)
vulnerability_db_notes = pd.DataFrame(vulnerability_db_notes)
vulnerability_db_threats = pd.DataFrame(vulnerability_db_threats)
vulnerability_db_score_set = pd.DataFrame(vulnerability_db_score_set)
vulnerability_db_acknowledgment = pd.DataFrame(vulnerability_db_acknowledgment)
vulnerability_db_revision = pd.DataFrame(vulnerability_db_revision)

print(vulnerability_db_notes)


      VulnerabilityFK                            Title         Type Ordinal  \
0                   0                 NIST NVD Details        Other       1   
1                   0                 NIST NVD Details        Other       1   
2                   0                      Description  Description       0   
3                   0                          Mariner          Tag      20   
4                   0                    cve@mitre.org          CNA      30   
...               ...                              ...          ...     ...   
1393              228                      Description  Description       0   
1394              228                              FAQ          FAQ      10   
1395              228                              FAQ          FAQ      10   
1396              228  Microsoft Edge (Chromium-based)          Tag      20   
1397              228                           Chrome          CNA      30   

                                                   